# Classification d'images avec SVM et caractéristiques HOG sur CIFAR-10

Ce notebook présente un pipeline complet pour la classification binaire d'images (chats vs chiens) en utilisant un classificateur SVM avec des caractéristiques HOG sur le jeu de données CIFAR-10. Chaque étape est expliquée ci-dessous, incluant le chargement des données, l'extraction des caractéristiques, l'entraînement du modèle, l'évaluation et la visualisation.

## Pourquoi choisir CIFAR-10 ?
- **Description** : CIFAR-10 contient 60 000 images RGB de 32x32 pixels dans 10 classes (avion, voiture, oiseau, chat, chien, etc.), avec 50 000 images d'entraînement et 10 000 de test.
- **Accessibilité** : Disponible via `tensorflow.keras.datasets.cifar10`, sans téléchargement manuel.
- **Pertinence** : La petite taille des images (32x32) est idéale pour l'extraction HOG, et le jeu de données supporte la classification binaire (chats vs chiens).

## Aperçu du pipeline
1. **Chargement et prétraitement** : Charger CIFAR-10, sélectionner les images de chats et chiens, convertir en niveaux de gris.
2. **Extraction HOG** : Calculer les descripteurs HOG pour chaque image.
3. **Préparation des données** : Créer des ensembles d'entraînement/test équilibrés, standardiser les caractéristiques.
4. **Entraînement SVM** : Utiliser un classificateur SVM linéaire.
5. **Évaluation** : Calculer l'exactitude, le rapport de classification et la matrice de confusion.
6. **Visualisation** : Afficher la matrice de confusion et des prédictions d'échantillons.

## Prérequis
Installez les bibliothèques nécessaires :
```bash
pip install numpy opencv-python scikit-learn scikit-image tensorflow matplotlib seaborn
```

In [1]:
# Importer les bibliothèques nécessaires
import numpy as np
import cv2
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.datasets import cifar10
import warnings
warnings.filterwarnings("ignore")  # Supprimer les avertissements pour un affichage propre

## Étape 1 : Extraction des caractéristiques HOG

La fonction `extract_hog_features` calcule les descripteurs HOG pour une image :
- **Entrée** : Une image RGB ou en niveaux de gris.
- **Traitement** : Convertit en niveaux de gris, extrait les caractéristiques HOG avec des paramètres spécifiés.
- **Paramètres** :
  - `orientations=8` : Nombre de bins pour les orientations de gradient.
  - `pixels_per_cell=(8, 8)` : Taille de chaque cellule (8x8 pixels).
  - `cells_per_block=(2, 2)` : Taille de chaque bloc (2x2 cellules) pour la normalisation.
- **Sortie** : Un vecteur de caractéristiques 1D (par exemple, 288 caractéristiques pour une image 32x32).

**Taille du vecteur de caractéristiques** (pour une image 32x32) :
- Cellules : `(32/8) x (32/8) = 4x4 = 16` cellules.
- Blocs : `(4-2+1) x (4-2+1) = 3x3 = 9` blocs.
- Caractéristiques par bloc : `2x2 cellules * 8 orientations = 32`.
- Total : `9 blocs * 32 = 288` caractéristiques.

In [2]:
# Fonction pour extraire les caractéristiques HOG d'une image
def extract_hog_features(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=8):
    # Convertir en niveaux de gris si l'image est RGB
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculer les caractéristiques HOG
    fd = hog(
        img,
        orientations=orientations,
        pixels_per_cell=pixels_per_cell,
        cells_per_block=cells_per_block,
        visualize=False
    )
    return fd

## Étape 2 : Chargement et prétraitement de CIFAR-10

La fonction `load_cifar10_binary` prépare le jeu de données pour la classification binaire :
- Charge CIFAR-10 avec `cifar10.load_data()`.
- Sélectionne les images de chats (étiquette 3) et de chiens (étiquette 5).
- Attribue des étiquettes binaires : 1 pour les chats, 0 pour les chiens.
- Limite à 1000 images par classe pour l'entraînement et 200 par classe pour le test pour équilibrer calcul et performance.
- Retourne des tableaux numpy pour les images et les étiquettes.

In [3]:
# Fonction pour charger CIFAR-10 pour la classification binaire (chats vs chiens)
def load_cifar10_binary(num_train_per_class=1000, num_test_per_class=200):
    # Charger le jeu de données CIFAR-10
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    
    # Étiquettes CIFAR-10 : 3 = chat, 5 = chien
    cat_label, dog_label = 3, 5
    X_train_binary, y_train_binary = [], []
    X_test_binary, y_test_binary = [], []
    
    # Sélectionner les images de chats et chiens pour l'entraînement
    cat_count, dog_count = 0, 0
    for i in range(len(X_train)):
        if y_train[i] == cat_label and cat_count < num_train_per_class:
            X_train_binary.append(X_train[i])
            y_train_binary.append(1)  # Chat = 1
            cat_count += 1
        elif y_train[i] == dog_label and dog_count < num_train_per_class:
            X_train_binary.append(X_train[i])
            y_train_binary.append(0)  # Chien = 0
            dog_count += 1
        if cat_count >= num_train_per_class and dog_count >= num_train_per_class:
            break
    
    # Sélectionner les images de chats et chiens pour le test
    cat_count, dog_count = 0, 0
    for i in range(len(X_test)):
        if y_test[i] == cat_label and cat_count < num_test_per_class:
            X_test_binary.append(X_test[i])
            y_test_binary.append(1)  # Chat = 1
            cat_count += 1
        elif y_test[i] == dog_label and dog_count < num_test_per_class:
            X_test_binary.append(X_test[i])
            y_test_binary.append(0)  # Chien = 0
            dog_count += 1
        if cat_count >= num_test_per_class and dog_count >= num_test_per_class:
            break
    
    return (np.array(X_train_binary), np.array(y_train_binary),
            np.array(X_test_binary), np.array(y_test_binary))

## Étape 3 : Visualisation des prédictions

La fonction `visualize_predictions` affiche des images de test avec leurs étiquettes réelles et prédites :
- Sélectionne aléatoirement 6 images de test.
- Affiche chaque image avec son étiquette réelle (Chat ou Chien) et prédite.
- Utilise Matplotlib pour un affichage en grille.

In [4]:
# Fonction pour visualiser les prédictions d'échantillons
def visualize_predictions(X_test, y_test, y_pred, num_samples=6):
    plt.figure(figsize=(12, 4))
    indices = np.random.choice(len(X_test), num_samples, replace=False)
    for i, idx in enumerate(indices):
        plt.subplot(2, num_samples//2, i+1)
        plt.imshow(X_test[idx])
        plt.title(f"Vrai : {'Chat' if y_test[idx] == 1 else 'Chien'}\nPrédit : {'Chat' if y_pred[idx] == 1 else 'Chien'}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

## Étape 4 : Pipeline principal

Le pipeline principal :
- Charge et prétraite les données.
- Extrait les caractéristiques HOG pour toutes les images.
- Standardise les caractéristiques pour améliorer les performances du SVM.
- Entraîne un classificateur SVM linéaire.
- Évalue le modèle avec l'exactitude, le rapport de classification et la matrice de confusion.
- Visualise les résultats.

**Performance attendue** :
- Exactitude : ~60-70% en raison de la complexité de CIFAR-10 (basse résolution, fonds variés).
- Améliorations possibles : ajuster les paramètres HOG, utiliser un noyau non linéaire, augmenter les données d'entraînement.

In [5]:
# Pipeline principal
# Définir une graine aléatoire pour la reproductibilité
np.random.seed(42)

# Charger et prétraiter les données
print("Chargement du jeu de données CIFAR-10...")
X_train, y_train, X_test, y_test = load_cifar10_binary(
    num_train_per_class=1000, num_test_per_class=200
)

# Extraire les caractéristiques HOG
print("Extraction des caractéristiques HOG...")
X_train_hog = [extract_hog_features(img) for img in X_train]
X_test_hog = [extract_hog_features(img) for img in X_test]

# Convertir en tableaux numpy
X_train_hog = np.array(X_train_hog)
X_test_hog = np.array(X_test_hog)

# Standardiser les caractéristiques
print("Standardisation des caractéristiques...")
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_test_hog = scaler.transform(X_test_hog)

# Entraîner le classificateur SVM
print("Entraînement du classificateur SVM...")
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train_hog, y_train)

# Évaluer le modèle
print("Évaluation du modèle...")
y_pred = svm.predict(X_test_hog)
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitude : {accuracy:.4f}")
print("\nRapport de classification :")
print(classification_report(y_test, y_pred, target_names=['Chien', 'Chat']))

# Afficher la matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Chien', 'Chat'], yticklabels=['Chien', 'Chat'])
plt.title('Matrice de confusion')
plt.xlabel('Prédit')
plt.ylabel('Vrai')
plt.show()

# Visualiser les prédictions d'échantillons
print("Visualisation des prédictions d'échantillons...")
visualize_predictions(X_test, y_test, y_pred, num_samples=6)

Chargement du jeu de données CIFAR-10...
 17301504/170498071 ━━━━━━━━━━━━━━━━━━━━ 2:11:16 51us/step

KeyboardInterrupt: 

## Notes et extensions

- **Amélioration des performances** :
  - Ajuster les paramètres HOG (par exemple, `pixels_per_cell=(4, 4)`, `orientations=9`).
  - Utiliser un noyau non linéaire (par exemple, `kernel='rbf'`) ou optimiser `C` avec `GridSearchCV`.
  - Augmenter les données d'entraînement (par exemple, 5000 images par classe).
  - Normaliser les images avant HOG (par exemple, `img = img / 255.0`).
- **Extension à la classification multi-classe** :
  - Inclure les 10 classes de CIFAR-10.
  - Utiliser `OneVsRestClassifier(SVC())` pour un SVM multi-classe.
- **Jeux de données alternatifs** :
  - **MNIST** : Chiffres manuscrits (28x28), via `tensorflow.keras.datasets.mnist`.
  - **Oxford-IIIT Pet** : Chats vs chiens, nécessite un téléchargement manuel.
  - **Fashion-MNIST** : Vêtements, via `tensorflow.keras.datasets.fashion_mnist`.
- **Dépannage** :
  - Réduire `num_train_per_class` en cas de problèmes de mémoire.
  - Vérifier la cohérence des tailles des caractéristiques HOG.

**Citation** : Krizhevsky, A. (2009). Learning Multiple Layers of Features from Tiny Images. [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)